# <span style="color:red; font-weight:bold; ">A clean and modern RangeViT implementation for SemanticKITTI in PyTorch 2.4</span>  

## <span style="font-weight:bold">1. DataLoader</span>

### 1.1 Dataset Structure
The dataset should be structured as follows:
```
sequences/
├── 00/
│   ├── preprocess/
│   │   ├── 000000.bin
│   │   ├── 000001.bin
├── 01/
│   ├── preprocess/
│   │   ├── 000000.bin
│   │   ├── 000001.bin
```



In [1]:
import torch.optim as optim
import torch
import numpy as np

import os
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm

from model.KITTISegmentationDataset import KITTISegmentationDataset
from model.RangeViTSegmentationModel import RangeViTSegmentationModel

from segmentation_models_pytorch.losses import FocalLoss, LovaszLoss


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = KITTISegmentationDataset('../sequences',['00', '01', '02', '03', '04', '05', '06', '07', '09', '10'], training=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

val_dataset = KITTISegmentationDataset('../sequences',['08'], training=False)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)



In [3]:
# Use torchmetrics or do manually
from torchmetrics.classification import MulticlassJaccardIndex
# create a metric and put it on gpu
metric = MulticlassJaccardIndex(num_classes=20, average=None, ignore_index=0).to(device)

num_classes = 20
in_channels = 9 # range, x, y, z, intensity, flag, R, G, B
num_epochs = 60
model = RangeViTSegmentationModel(n_classes=num_classes, in_channels=in_channels).to(device)
# criterion = LovaszLoss(mode='multiclass', ignore_index=0, per_image=False)
focal = FocalLoss(mode='multiclass', ignore_index=0)
lovasz = LovaszLoss(mode='multiclass', ignore_index=0, per_image=False)
def criterion(outputs, targets):
    return focal(outputs, targets) + lovasz(outputs, targets)
optimizer = optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.01, betas=(0.9, 0.999))

from torch.optim.lr_scheduler import CosineAnnealingLR
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0)

In [4]:
def train_one_epoch(model, loader, optimizer, criterion, metric,epoch):
    model.train()
    total_loss = 0.0
    metric.reset()  # Reset the IoU metric for the next epoch
    batch_bar = tqdm(loader, desc=f"Training Epoch {epoch+1}", leave=False)
    for imgs, labels in batch_bar:
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        preds = outputs.argmax(dim=1)
        metric.update(preds, labels)
        ious = metric.compute()
        mean_iou = torch.mean(ious[ious != 0])
        loss.backward()
        optimizer.step()
        batch_bar.set_postfix(loss=loss.item(), mIoU=mean_iou.item())
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}] Loss: {total_loss/len(loader):.4f}, mIoU: {mean_iou.item():.4f}")


In [5]:
def eval_model(model, loader, criterion, metric):
    model.eval()
    total_loss = 0.0
    metric.reset()  # Reset the IoU metric for the evaluation
    with torch.no_grad():
        batch_bar = tqdm(loader, desc="Evaluating", leave=False)
        for imgs, labels in batch_bar:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            preds = outputs.argmax(dim=1)
            metric.update(preds, labels)
            ious = metric.compute()
            mean_iou = torch.mean(ious[ious != 0])
            batch_bar.set_postfix(loss=loss.item(), mIoU=mean_iou.item())
            total_loss += loss.item()
    print(f"Evaluation Loss: {total_loss/len(loader):.4f}, mIoU: {mean_iou.item():.4f}")


In [6]:
# Load the model if you have a pre-trained one
pretrain_path = 'range_vit_segmentation.pth'
if os.path.exists(pretrain_path):
    print(f"Loading pre-trained model from {pretrain_path}")
    model.load_state_dict(torch.load('range_vit_segmentation.pth'))

Loading pre-trained model from range_vit_segmentation.pth


In [7]:
# dirty
# 1. load the ../model_skitti_trainval_cs_init_h256.pth
# 2. copy model.rangevit.encoder.blocks to the model state_dict['backbone.blocks']
# 3. freeze the backbone
state_dict = torch.load("../model_skitti_trainval_cs_init_h256.pth", map_location="cpu")
# Filter keys for blocks
blocks_state_dict = {}
for k, v in state_dict['model'].items():
    if 'rangevit.encoder.blocks' in k:
        # Remove the prefix so keys match model.backbone.blocks
        new_k = k.replace('rangevit.encoder.blocks.', '')
        blocks_state_dict[new_k] = v

# Load into model
model.backbone.blocks.load_state_dict(blocks_state_dict, strict=False)
for param in model.backbone.blocks.parameters():
    param.requires_grad = False

In [8]:
# Freeze multi-head attention layers, fine-tune LayerNorm and MLP layers in ViT encoder
# for name, module in model.backbone.named_modules():
#     # Freeze all MultiheadAttention layers
#     if isinstance(module, torch.nn.modules.activation.MultiheadAttention) or 'attn' in name:
#         for param in module.parameters():
#             param.requires_grad = False
#     # Unfreeze LayerNorm and MLP layers
#     if isinstance(module, torch.nn.LayerNorm) or 'mlp' in name:
#         for param in module.parameters():
#             param.requires_grad = True


In [9]:
### Train the model
best_val_mIoU = 0.0
# Training loop
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    train_one_epoch(model, loader, optimizer, criterion, metric,epoch)
    if epoch % 5 == 0: # Evaluate every 5 epochs
        eval_model(model, val_loader, criterion, metric)
        ious = metric.compute()
        current_val_mIoU = torch.mean(ious[ious != 0]).item()
        if current_val_mIoU > best_val_mIoU:
            best_val_mIoU = current_val_mIoU
            torch.save(model.state_dict(), 'range_vit_segmentation.pth')
    scheduler.step()


Epochs:   0%|          | 0/60 [00:00<?, ?it/s]

Training Epoch 1:   0%|          | 0/598 [00:00<?, ?it/s]

Epoch [1] Loss: 1.0559, mIoU: 0.3091


Evaluating:   0%|          | 0/255 [00:00<?, ?it/s]

Evaluation Loss: 1.0796, mIoU: 0.2633


...

Epoch [58] Loss: 0.7510, mIoU: 0.5231


Training Epoch 59:   0%|          | 0/598 [00:00<?, ?it/s]

Epoch [59] Loss: 0.7489, mIoU: 0.5223


Training Epoch 60:   0%|          | 0/598 [00:00<?, ?it/s]

Epoch [60] Loss: 0.7485, mIoU: 0.5229


In [10]:
# Validation with the best model
model = RangeViTSegmentationModel(n_classes=num_classes, in_channels=in_channels).to(device)
model.load_state_dict(torch.load('range_vit_segmentation.pth'))
eval_model(model, val_loader, criterion, metric)


Evaluating:   0%|          | 0/255 [00:00<?, ?it/s]

Evaluation Loss: 0.9043, mIoU: 0.3334


In [11]:
# print structure of model
# print(model)